<a href="https://colab.research.google.com/github/kevinhhl/options-pricing-tools-and-trading-strategies/blob/main/Black_Scholes_Merton_Model_Part2_Position_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview

Back in [Part 1](https://github.com/kevinhhl/options-pricing-tools-and-trading-strategies/blob/main/Black_Scholes_Merton_Model_Part1_Screening_YF_for_theoretical_edges.ipynb), we implemented the Black Scholes Model (BSM) and applied it to data provided by Yahoo. We had a summary table to list out the theoretical values and Greeks for each of the individual options in the options chain.

When constructing a portfolio of options, we need to make notes of those options that we want to include in our positions. We need to store the option data in hierarchical forms.



# Implementation

Here is the blueprint for three different levels of datatypes that will be implementing to store options data. Programming it this way will allow us to develop further tools to perform risk analysis for options portfolios from both top-down and bottom-up perspectives.

<br>

| Hierarchy & name of datatype | Description |
|--|--|
| Level 1: Contract| A datatype to store data points attributed to an individual option (ie. maturity date, strike price, call/put).  |
| Level 2: Leg | A data structure that stores (1) a single Contract (2) an instance of a BSM, and (3) the outputs from BSM|
| Level 3: Position | A collection of multiple Legs |





In [ ]:
!pip install yahoo_fin
from yahoo_fin import options

!pip install yfinance
import yfinance as yf

import math
import time
import pandas as pd
from datetime import date
from scipy.stats import norm

#### class BSM():

In [ ]:
class BSM:
  
  def __init__(self, x, s,t,r,sigma):
    self.d1, self.d2 = None, None
    self.tv_call, self.delta_call , self.gamma_call, self.vega_call, self.theta_call, self.rho_call = None, None, None, None, None, None
    self.tv_put, self.delta_put, self.gamma_put, self.vega_put, self.theta_put, self.rho_put = None, None, None, None, None, None

    self.x = x
    self.s = s
    self.t = t
    self.r = r
    self.sigma = sigma
    self.calc()

  def calc(self):
    ''' Calculates all the theoretical and Greeks values for call and put. 
        Need to call calc() whenever new input is provided. 
    '''
    _a = math.log(self.s/ self.x)
    _b = (self.r+self.sigma**2/2)*self.t
    self.d1 = (_a+_b)/self.sigma*math.sqrt(self.t)
    self.d2 = self.d1 - self.sigma * math.sqrt(self.t)
    
    # Call: 
    self.tv_call    = self.s * norm.cdf(self.d1) - self.x*math.exp(-self.r*self.t)*norm.cdf(self.d2)
    self.delta_call = norm.cdf(self.d1)
    self.gamma_call = norm.pdf(self.d1)/(self.s*self.sigma*math.sqrt(self.t))
    self.vega_call  = 0.01*(self.s*norm.pdf(self.d1)*math.sqrt(self.t))
    self.theta_call = 0.01*(-(self.s*norm.pdf(self.d1)*self.sigma)/(2*math.sqrt(self.t)) - self.r*self.x*math.exp(-self.r*self.t)*norm.cdf(self.d2))
    self.rho_call   = 0.01*(self.x*self.t*math.exp(-self.r*self.t)*norm.cdf(self.d2))
    
    # Put: 
    self.tv_put    = self.x * math.exp(-self.r*self.t)-self.s+self.tv_call
    self.delta_put = -norm.cdf(-self.d1)
    self.gamma_put = norm.pdf(self.d1)/(self.s*self.sigma*math.sqrt(self.t))
    self.vega_put  = 0.01*(self.s*norm.pdf(self.d1)*math.sqrt(self.t))
    self.theta_put = 0.01*(-(self.s*norm.pdf(self.d1)*self.sigma)/(2*math.sqrt(self.t)) + self.r*self.x*math.exp(-self.r*self.t)*norm.cdf(-self.d2))
    self.rho_put   = 0.01*(-self.x*self.t*math.exp(-self.r*self.t)*norm.cdf(-self.d2))



#### class contract():

In [ ]:
class contract:

  def __init__(self, expiry_date, strike, contracttype, quoted_price):
    self.multiple = 100
    self.strike = strike
    self.contracttype=contracttype
    self.expiry_date = expiry_date
    self.price = quoted_price

  def __str__(self):
    return "{}_{}_{} @{}".format(self.expiry_date, self.strike, self.contracttype, self.price)

#### class leg():

In [ ]:
class leg:

  def __init__(self, side, contract, size, date_today, s,r,sigma):
    self.size=size
    leg.size = size
    self.side = side
    t = (contract.expiry_date-date_today).days/365
    self.contract = contract
    self.model = BSM(contract.strike, s,t,r,sigma)

    self.delta = None
    self.rho = None
    self.vega = None
    self.theta = None   
    self.gamma = None
    self.edge = 0
    self.premium = 0

    # int _coefficient, has three purposes:
    #   (1) +/-ve Delta for long call/put, but it's reversed from seller's perspective  
    #   (2) If buyer of contract:   +ve vega, -ve theta, +ve gamma
    #   (3) If seller of contract:  -ve vega, +ve theta, -ve gamma
    _coefficient = None
    if self.side == "long":
      _coefficient = 1
    elif self.side == "short":
      _coefficient = -1
    
    if contract.contracttype=="call":
      if self.side == "long":
        self.edge = (self.model.tv_call-self.contract.price) * size 
      elif self.side == "short":
        self.edge = (self.contract.price-self.model.tv_call) * size 
      self.delta = self.model.delta_call * size  * _coefficient
      self.rho = self.model.rho_call * size      * _coefficient
      self.vega = self.model.vega_call * size    * _coefficient
      self.gamma = self.model.gamma_call * size  * _coefficient
      self.theta = self.model.theta_call * size  * _coefficient
    
    elif contract.contracttype=="put":
      if self.side == "long":
        self.edge = (self.model.tv_put-self.contract.price) * size 
      elif self.side == "short":
        self.edge = (self.contract.price-self.model.tv_put) * size 
      self.delta = self.model.delta_put * size   * _coefficient
      self.rho = self.model.rho_put * size       * _coefficient
      self.vega = self.model.vega_put * size     * _coefficient
      self.gamma = self.model.gamma_put * size   * _coefficient
      self.theta = self.model.theta_put * size   * _coefficient
    
    self.premium = contract.price * size * -_coefficient

  def __str__(self):
    n_round= 4 
    return "{} [x{}]: Δ={},𝚪={},Θ={},V={},⍴={}".format(str(self.contract),self.size, \
      self.delta.round(n_round),self.gamma.round(n_round),self.theta.round(n_round),self.vega.round(n_round),self.rho.round(n_round))


### class position():

In [ ]:
class position:

  def __init__(self):
    self.list_leg = []
    self.total_delta = 0
    self.total_gamma = 0
    self.total_vega = 0
    self.total_theta = 0
    self.total_rho = 0
    self.total_premium = 0
    self.total_th_edge = 0

  def get_summary(self, rounding=4) -> dict:
    ''' Returns dict of {string:float}
    '''
    return {"Total delta" : round(self.total_delta,rounding),
            "Total gamma" : round(self.total_gamma,rounding),
            "Total vega"  : round(self.total_vega,rounding),
            "Total theta" : round(self.total_theta,rounding),
            "Total rho"   :round(self.total_rho,rounding),
            "Premium (paid)/received" : round(self.total_premium,rounding),
            "Th. Edge"    : round(self.total_th_edge,rounding),
            }
  
  def add_leg(self, leg):
    ''' @param leg
    '''
    size = leg.size
    self.total_premium += leg.premium
    self.total_th_edge += leg.edge
    self.list_leg.append(leg)
    
    self.total_delta += leg.delta
    self.total_gamma += leg.gamma
    self.total_vega += leg.vega
    self.total_theta += leg.theta
    self.total_rho += leg.rho


### Other Methods:

In [ ]:
def populate_dict(yf_chain, contracttype):
  ''' @param Object yf_chain : from options.get_options_chain(symbol, date_expire_str)["calls" or "puts"]
      @param string contracttype : either "call" or "put
      Returns a dict of {float strike price : contract object}
  '''
  out = {}
  for i in range(len(yf_chain)):
    x = yf_chain["Strike"][i]
    out[x] = contract(date_expire, x, contracttype, yf_chain["Last Price"][i])
  return out


#Application:
😎 emoji = Requires user's attention.  

Setting up our model:


In [ ]:
# 😎 Manual inputs:
symbol                   = "TSLA" 
sigma                    = 0.70 
riskfree_rate            = math.e**(.0395)-1
date_today               = date(2023,2,22) 
date_expire              = date(2023,3,17) 

In [ ]:
ttm = (date_expire-date_today).days/365

# Confirming that the expiration date is valid.
exp_dates = options.get_expiration_dates(symbol)
date_expire_str = date_expire.strftime("%B %d, %Y") 
assert date_expire_str in exp_dates 

# Obtaining the recent close priceticker_yahoo = yf.Ticker(symbol)
ticker_yahoo = yf.Ticker(symbol)
data = ticker_yahoo.history()
crnt_price = data['Close'].iloc[-1]
crnt_price

199.72000122070312

---
##Option Chain Analysis:

In [ ]:
original_chain_call = options.get_options_chain(symbol, date_expire_str)["calls"]
original_chain_put = options.get_options_chain(symbol, date_expire_str)["puts"]

calls = populate_dict(original_chain_call, "call")
puts = populate_dict(original_chain_put, "put")

## Position Analysis

**Long straddle:**

In [ ]:
pos = position()
leg1 = leg(side="long", contract=calls[210], size=1, date_today=date_today, s=crnt_price, r=riskfree_rate, sigma=sigma)
leg2 = leg(side="long", contract=puts[210], size=1, date_today=date_today, s=crnt_price, r=riskfree_rate, sigma=sigma)
pos.add_leg(leg1)
pos.add_leg(leg2)
for e in pos.list_leg:
  print(e)
pos.get_summary()

2023-03-17_210.0_call @9.8 [x1]: Δ=0.4954,𝚪=0.0114,Θ=-1.1468,V=0.2,⍴=0.0562
2023-03-17_210.0_put @21.3 [x1]: Δ=-0.5046,𝚪=0.0114,Θ=-1.0624,V=0.2,⍴=-0.0758


{'Total delta': -0.0092,
 'Total gamma': 0.0227,
 'Total vega': 0.4,
 'Total theta': -2.2092,
 'Total rho': -0.0196,
 'Premium (paid)/received': -31.1,
 'Th. Edge': -1.8242}

**Short straddle:**

In [ ]:
pos = position()
leg1 = leg(side="short", contract=calls[210], size=1, date_today=date_today, s=crnt_price, r=riskfree_rate, sigma=sigma)
leg2 = leg(side="short", contract=puts[210], size=1, date_today=date_today, s=crnt_price, r=riskfree_rate, sigma=sigma)
pos.add_leg(leg1)
pos.add_leg(leg2)
for e in pos.list_leg:
  print(e)
pos.get_summary()

2023-03-17_210.0_call @9.8 [x1]: Δ=-0.4954,𝚪=-0.0114,Θ=1.1468,V=-0.2,⍴=-0.0562
2023-03-17_210.0_put @21.3 [x1]: Δ=0.5046,𝚪=-0.0114,Θ=1.0624,V=-0.2,⍴=0.0758


{'Total delta': 0.0092,
 'Total gamma': -0.0227,
 'Total vega': -0.4,
 'Total theta': 2.2092,
 'Total rho': 0.0196,
 'Premium (paid)/received': 31.1,
 'Th. Edge': 1.8242}

### Validating the results:

Example above: Long & Short Strangle

In theory, the positions should have the following +ve / -ve of Greeks. 
 
|Spread|Delta|Gamma|Theta|Vega|Downside|Upside|
|-|-|-|-|-|-|-|
|Long straddle|0|+ve|-ve|+ve|Unlimited reward|Unlimited reward|
|Short straddle|0|-ve|+ve|-ve|Unlimited risk|Unlimited risk|

In a perfect world, deltas for ATM calls and puts are exactly 0.5 and -0.5, respectively; we will be delta neutral if we buy/sell straddles ATM. In this case, we are buying and selling straddles that are near-the-money, the net Delta should be close to zero.

## More notes:

***Summary of common volatility spreads [1]:***

Profitable when: +ve Delta, -ve Gamma, +ve Vega

|Spread|Delta|Gamma|Theta|Vega|Downside|Upside|
|-|-|-|-|-|-|-|
|Long straddle|0|+ve|-ve|+ve|N/A|Unlimited reward|
|Long strangle|0|+ve|-ve|+ve|N/A|Unlimited reward|
|Short butterfly|0|+ve|-ve|+ve|N/A|Limited reward|
|Short condor|0|+ve|-ve|+ve|N/A|Limited reward|


Profitable when: -ve Delta, +ve Gamma, -ve Vega

|Spread|Delta|Gamma|Theta|Vega|Downside|Upside|
|-|-|-|-|-|-|-|
|Short straddle|0|-ve|+ve|-ve|Unlimited risk|N/A|
|Short strangle|0|-ve|+ve|-ve|Unlimited risk|N/A|
|Long butterfly|0|-ve|+ve|-ve|Limited risk|N/A|
|Long condor|0|-ve|+ve|-ve|Limited risk|N/A|


**Observations**: Straddle, strangle, butterfly, and condor are very similar in terms of +ve/-ve of Gamma, Theta, and Vega that the trader would like the market to give to the positions.  
<br>

---
*References:*

[1] Natenberg, Sheldon. <i>Chapter 11, Option Volatility and Pricing, Second Edition</i>. McGraw-Hill Edu., 2015.


